In [74]:
import os
import pandas as pd
import numpy as np

carpeta_tweets = "./tweets"

categorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]

dataframes_por_archivo = {}

for nombre_archivo in os.listdir(carpeta_tweets):
    if nombre_archivo.endswith(".txt"):
        ruta_archivo = os.path.join(carpeta_tweets, nombre_archivo)

        datos = []
        with open(ruta_archivo, mode='r', encoding='utf-8') as archivo:
            for linea in archivo:
                partes = linea.strip().split("#############")
                if len(partes) != 3:
                    continue

                texto, etiquetas, tweet_padre = partes

                etiquetas_set = set(et.strip() for et in etiquetas.split(",") if et.strip())
                if not etiquetas_set:
                    etiquetas_set = {"Incensurable"}

                fila = {
                    "Tweet": texto,
                    "Tweet_Padre": tweet_padre
                }

                for cat in categorias_posibles:
                    fila[cat] = int(cat in etiquetas_set)

                datos.append(fila)

        nombre_df = os.path.splitext(nombre_archivo)[0]
        df = pd.DataFrame(datos)
        dataframes_por_archivo[nombre_df] = df

In [75]:
df_unificado = pd.concat(dataframes_por_archivo.values(), ignore_index=True)
df_unificado.head()

,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,@TomasVodanovic tiene convertido Maipú en un ...,Policial Homicidio se registró la tarde de est...,0,0,1,0
1,Adivinemos..... venekia!,"Esta noche en #ReportajesT13 : Fiestas, máqu...",0,0,1,0
2,Y por q mejor no expulsan luego a los inmigran...,Nacidos entre 1971 y 1981 deben vacunarse con...,0,0,1,0
3,Balazos y se acaba esta mierda!,Usan quebradas para esconderse y vigilar casa...,1,0,0,0
4,Hay que poner guardias armados y eliminar esas...,Pero que los confundan con conejos...,1,0,0,0


In [76]:
df_unificado.shape[0]

1008

In [77]:
nulos_tweet = df_unificado['Tweet'].isna().sum()
vacíos_tweet = (df_unificado['Tweet'].astype(str).str.strip() == "").sum()
nulos_tweet_padre = df_unificado['Tweet_Padre'].isna().sum()
vacíos_tweet_padre = (df_unificado['Tweet_Padre'].astype(str).str.strip() == "").sum()

print(f"Total de Tweets nulos: {nulos_tweet + vacíos_tweet}")
print(f"Total de Tweets padres nulos: {nulos_tweet_padre + vacíos_tweet_padre}")


Total de Tweets nulos: 2
Total de Tweets padres nulos: 10


In [78]:
df_unificado['Tweet_Padre'] = df_unificado['Tweet_Padre'].fillna("SinTweetPadre")
df_unificado['Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(
    lambda x: "SinTweetPadre" if isinstance(x, str) and x.strip() == "" else x
)

df_unificado = df_unificado.dropna(subset=['Tweet'])
df_unificado = df_unificado[df_unificado['Tweet'].str.strip() != ""]


df_unificado = df_unificado.drop_duplicates()


In [79]:
duplicados = df_unificado.duplicated(subset='Tweet', keep=False)

num_duplicados = duplicados.sum()
print(f"Número total de tweets con texto repetido: {num_duplicados}")

tweets_duplicados = df_unificado[duplicados].sort_values('Tweet')
display(tweets_duplicados)


Número total de tweets con texto repetido: 6


,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
551,La xenofobia es un deber estético y ético. ...,SinTweetPadre,1,0,1,0
951,La xenofobia es un deber estético y ético. ...,SinTweetPadre,0,0,1,0
56,No queremos más venecos en Chile,Repito: que puto asco . Tienen algo decente p...,0,0,1,0
634,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0
20,Wooo!! no había cacha'o .. veneco culiao cínico!!,Si alguien reconoce su celular ubicar al venec...,0,0,1,0
677,Wooo!! no había cacha'o .. veneco culiao cínico!!,SinTweetPadre,0,0,1,0


In [80]:
df_unificado = df_unificado.drop([951, 634, 677])
df_unificado = df_unificado.reset_index(drop=True) 

duplicados = df_unificado.duplicated(subset='Tweet', keep=False)

num_duplicados = duplicados.sum()
print(f"Número total de tweets con texto repetido: {num_duplicados}")


Número total de tweets con texto repetido: 0


In [81]:
df_unificado['ID'] = range(1, len(df_unificado) + 1)

columnas_ordenadas = ['ID'] + [col for col in df_unificado.columns if col != 'ID']
df_unificado = df_unificado[columnas_ordenadas]

df_unificado.head()


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,1,@TomasVodanovic tiene convertido Maipú en un ...,Policial Homicidio se registró la tarde de est...,0,0,1,0
1,2,Adivinemos..... venekia!,"Esta noche en #ReportajesT13 : Fiestas, máqu...",0,0,1,0
2,3,Y por q mejor no expulsan luego a los inmigran...,Nacidos entre 1971 y 1981 deben vacunarse con...,0,0,1,0
3,4,Balazos y se acaba esta mierda!,Usan quebradas para esconderse y vigilar casa...,1,0,0,0
4,5,Hay que poner guardias armados y eliminar esas...,Pero que los confundan con conejos...,1,0,0,0


In [82]:
texto_a_id = dict(zip(df_unificado['Tweet'], df_unificado['ID']))

def obtener_id_padre(texto_padre):
    if texto_padre == "SinTweetPadre":
        return np.nan
    return texto_a_id.get(texto_padre, "X")

df_unificado['ID_Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(obtener_id_padre)

num_casos_X = (df_unificado['ID_Tweet_Padre'] == "X").sum()
print(f"Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): {num_casos_X}")

df_unificado.head()


Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): 241


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable,ID_Tweet_Padre
0,1,@TomasVodanovic tiene convertido Maipú en un ...,Policial Homicidio se registró la tarde de est...,0,0,1,0,X
1,2,Adivinemos..... venekia!,"Esta noche en #ReportajesT13 : Fiestas, máqu...",0,0,1,0,X
2,3,Y por q mejor no expulsan luego a los inmigran...,Nacidos entre 1971 y 1981 deben vacunarse con...,0,0,1,0,X
3,4,Balazos y se acaba esta mierda!,Usan quebradas para esconderse y vigilar casa...,1,0,0,0,X
4,5,Hay que poner guardias armados y eliminar esas...,Pero que los confundan con conejos...,1,0,0,0,X


In [83]:
df_unificado.shape[0]

999

In [84]:
df_unificado.to_csv("tweets_limpios.csv", index=False, encoding='utf-8')

In [85]:
"""
import ipywidgets as widgets
from IPython.display import display, clear_output

tweets_padres_faltantes = df_unificado[df_unificado['ID_Tweet_Padre'] == "X"]['Tweet_Padre'].unique().tolist()

nuevos_tweets = []

categorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]

index = 0

texto_output = widgets.Output()
dropdown_categorias = widgets.SelectMultiple(
    options=categorias_posibles,
    description='Categorías:',
    style={'description_width': 'initial'}
)
boton_guardar = widgets.Button(description="Guardar y siguiente")
boton_omitir = widgets.Button(description="Omitir")
info_label = widgets.Label()

def mostrar_tweet():
    with texto_output:
        clear_output()
        if index < len(tweets_padres_faltantes):
            print(f"Tweet padre no encontrado ({index + 1}/{len(tweets_padres_faltantes)}):")
            print(tweets_padres_faltantes[index])
        else:
            print("¡Todos los tweets padres han sido procesados!")

def guardar_tweet(b):
    global index
    if index < len(tweets_padres_faltantes):
        texto = tweets_padres_faltantes[index]
        categorias = list(dropdown_categorias.value)
        etiquetas = categorias if categorias else ["Incensurable"]
        nuevo_id = df_unificado['ID'].max() + 1

        fila = {
            "ID": nuevo_id,
            "Tweet": texto,
            "Tweet_Padre": "SinTweetPadre",
            "ID_Tweet_Padre": np.nan
        }
        for cat in categorias_posibles:
            fila[cat] = int(cat in etiquetas)

        nuevos_tweets.append(fila)
        index += 1
        mostrar_tweet()

def omitir_tweet(b):
    global index
    index += 1
    mostrar_tweet()

boton_guardar.on_click(guardar_tweet)
boton_omitir.on_click(omitir_tweet)

mostrar_tweet()
display(texto_output, dropdown_categorias, boton_guardar, boton_omitir, info_label)
"""


'\nimport ipywidgets as widgets\nfrom IPython.display import display, clear_output\n\ntweets_padres_faltantes = df_unificado[df_unificado[\'ID_Tweet_Padre\'] == "X"][\'Tweet_Padre\'].unique().tolist()\n\nnuevos_tweets = []\n\ncategorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]\n\nindex = 0\n\ntexto_output = widgets.Output()\ndropdown_categorias = widgets.SelectMultiple(\n    options=categorias_posibles,\n    description=\'Categorías:\',\n    style={\'description_width\': \'initial\'}\n)\nboton_guardar = widgets.Button(description="Guardar y siguiente")\nboton_omitir = widgets.Button(description="Omitir")\ninfo_label = widgets.Label()\n\ndef mostrar_tweet():\n    with texto_output:\n        clear_output()\n        if index < len(tweets_padres_faltantes):\n            print(f"Tweet padre no encontrado ({index + 1}/{len(tweets_padres_faltantes)}):")\n            print(tweets_padres_faltantes[index])\n        else:\n            print("¡Todos los tweets padres han 

In [86]:
"""
df_unificado = pd.concat([df_unificado, pd.DataFrame(nuevos_tweets)], ignore_index=True)

texto_a_id = dict(zip(df_unificado['Tweet'], df_unificado['ID']))

df_unificado['ID_Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(
    lambda x: np.nan if x == "SinTweetPadre" else texto_a_id.get(x, "X")
)

print(f"Tweets con 'X' restantes: {(df_unificado['ID_Tweet_Padre'] == 'X').sum()}")

"""

'\ndf_unificado = pd.concat([df_unificado, pd.DataFrame(nuevos_tweets)], ignore_index=True)\n\ntexto_a_id = dict(zip(df_unificado[\'Tweet\'], df_unificado[\'ID\']))\n\ndf_unificado[\'ID_Tweet_Padre\'] = df_unificado[\'Tweet_Padre\'].apply(\n    lambda x: np.nan if x == "SinTweetPadre" else texto_a_id.get(x, "X")\n)\n\nprint(f"Tweets con \'X\' restantes: {(df_unificado[\'ID_Tweet_Padre\'] == \'X\').sum()}")\n\n'